
<h1> Trabajo práctico (3 puntos): Piedra Hoja Tijeras</h1>

Queremos desarrollar una máquina capaz de ganar al juego "Piedra Hoja Tijeras" anticipando el gesto del adversario. Suponemos que tenemos acceso a datos de sensores EMG (electromigrafía) que miden en tiempo real la actividad muscular del adversario.

Tenemos a nuestra disposición un dataset que contiene observaciones de 3 tipos de gestos: piedra (0), hoja (1) y tijeras (2). Por cada observación, tenemos acceso a 8 sensores musculares y se recopila 8 mediciones de cada sensor justo antes de que el jugador revele su gesto. Es decir, tenemos 64 mediciones por cada observación.



Queremos aprender un modelo de clasificación capaz de predecir que gesto va a hacer el jugador.

1)Desarrollar un script python para entrenar y evaluar al menos 3 modelos de clasificación distintos, limpiando o transformando los datos si lo estiman necesario.

2)¿Qué precisión y recall obtiene?

3)¿Cómo se comparta los puntajas de precisión y recall según el tamaño del dataset de entrenamiento? Probar con varios tamaño de dataset de entrenamiento y graficar las curvas de Precicion o Recall según el tamaño del dataset de entrenamiento. ¿Qué observan?



In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

dataset = pd.read_csv("prueba2/datos.csv")
dataset_clean = dataset.drop(['S1M2','S2M2','S3M2','S4M2','S5M2','S6M2','S7M2','S8M2',
                              'S1M3','S2M3','S3M3','S4M3','S5M3','S6M3','S7M3','S8M3',
                              'S1M4','S2M4','S3M4','S4M4','S5M4','S6M4','S7M4','S8M4',
                              'S1M5','S2M5','S3M5','S4M5','S5M5','S6M5','S7M5','S8M5',
                              'S1M6','S2M6','S3M6','S4M6','S5M6','S6M6','S7M6','S8M6',
                              'S1M7','S2M7','S3M7','S4M7','S5M7','S6M7','S7M7','S8M7',
                              'S1M8','S2M8','S3M8','S4M8','S5M8','S6M8','S7M8','S8M8'],1)

#Tenía pensado sacar un promedio de las 8 muestras para cada sensor y asi tener 8 datos pero no supe como hacerlo


dataset_clean[:8]

,S1M1,S2M1,S3M1,S4M1,S5M1,S6M1,S7M1,S8M1,Clase
0,26.0,4.0,5.0,8.0,-1.0,-13.0,-109.0,-66.0,0
1,-47.0,-6.0,-5.0,-7.0,13.0,-1.0,35.0,-10.0,0
2,-19.0,-8.0,-8.0,-8.0,-21.0,-6.0,-79.0,12.0,0
3,2.0,3.0,0.0,2.0,0.0,22.0,106.0,-14.0,0
4,6.0,0.0,0.0,-2.0,-14.0,10.0,-51.0,5.0,0
5,15.0,-5.0,-5.0,-15.0,12.0,-22.0,-38.0,36.0,0
6,-12.0,-5.0,-1.0,4.0,-16.0,-17.0,-69.0,-16.0,0
7,43.0,0.0,-2.0,6.0,11.0,26.0,51.0,27.0,0


In [2]:
dataset_clean.isnull().sum()

S1M1     0
S2M1     0
S3M1     0
S4M1     0
S5M1     0
S6M1     0
S7M1     0
S8M1     0
Clase    0
dtype: int64

In [3]:
dataset_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8756 entries, 0 to 8755
Data columns (total 9 columns):
S1M1     8756 non-null float64
S2M1     8756 non-null float64
S3M1     8756 non-null float64
S4M1     8756 non-null float64
S5M1     8756 non-null float64
S6M1     8756 non-null float64
S7M1     8756 non-null float64
S8M1     8756 non-null float64
Clase    8756 non-null int64
dtypes: float64(8), int64(1)
memory usage: 615.7 KB


## 2. Aprendizaje de un modelo de clasificación (Regresión logistica)

In [4]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn

from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report

In [5]:
X = dataset_clean.ix[:,(0,1,2,3,4,5,6,7)].values
y = dataset_clean.ix[:,8].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=25)
print(X_test)

[[ -4. -31.  10. ...  -1.  -1.  -2.]
 [ -6.  -9.  -5. ... -15.  -9. -35.]
 [ 83.   8.  -6. ... -41. -38. -21.]
 ...
 [  9.   6.  -2. ... -26. -58. -10.]
 [-14.  -7.   1. ...  15.   1.  -1.]
 [  6.  18.   2. ...  44.  22. -32.]]


In [7]:
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [8]:
print("coeficientes del modelo: "+str(LogReg.coef_))
print("intercept: "+str(LogReg.intercept_))

coeficientes del modelo: [[ 7.77280082e-04  8.46856811e-04 -5.65727729e-04  1.58074315e-03
   8.31061243e-04  1.54589308e-03 -3.61953251e-03  3.54524980e-03]
 [ 1.94460955e-03 -3.50697708e-04  2.27056534e-03 -2.17027578e-03
  -1.57415314e-03  8.00454626e-05  1.97849888e-03 -2.88086475e-03]
 [-2.73654788e-03 -4.65234679e-04 -1.72419489e-03  6.41276315e-04
   7.56730204e-04 -1.61508095e-03  1.61852862e-03 -6.29238367e-04]]
intercept: [-0.71144316 -0.68346229 -0.68631729]


In [9]:
#Predicciones
y_pred = LogReg.predict(X_test)
y_pred

array([1, 1, 1, ..., 0, 2, 1])

In [10]:
#Evaluación del rendimiento del clasificador
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
confusion_matrix

array([[326, 274, 290],
       [178, 305, 365],
       [220, 307, 362]])

In [11]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.37      0.40       890
           1       0.34      0.36      0.35       848
           2       0.36      0.41      0.38       889

   micro avg       0.38      0.38      0.38      2627
   macro avg       0.38      0.38      0.38      2627
weighted avg       0.38      0.38      0.38      2627



## 2. Aprendizaje de un modelo de clasificación de tipo árbol de decisión 

In [18]:
X = dataset.iloc[0:len(dataset),0:13]
y = dataset.iloc[0:len(dataset),13]

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=25)

from sklearn import tree
#Entrenamiento
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)

In [20]:
#Predicciones
y_pred = clf.predict(X_test)
#Evaluación del rendimiento del clasificador
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
#Print de la matriz de confusión
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
              precision    recall  f1-score   support

      -128.0       0.00      0.00      0.00         1
      -110.0       0.00      0.00      0.00         0
       -93.0       0.00      0.00      0.00         0
       -91.0       0.00      0.00      0.00         0
       -84.0       0.00      0.00      0.00         2
       -82.0       0.00      0.00      0.00         0
       -81.0       0.00      0.00      0.00         1
       -80.0       0.00      0.00      0.00         1
       -79.0       0.00      0.00      0.00         1
       -78.0       0.00      0.00      0.00         1
       -77.0       0.00      0.00      0.00         2
       -76.0       0.00      0.00      0.00         3
       -75.0       0.00      0.00      0.00         1
       -74.0       0.00      0.00      0.00         1
       -73.0       0.00      0.00      0.00         1
       -72.0   

In [22]:
from sklearn.externals.six import StringIO
import pydot #sudo apt install python3-pydot graphviz

dot_data = StringIO()

features=['S1','S2','S3','S4','S5','S6''S7','S8']
classes=['0','1','2']
tree.export_graphviz(clf,out_file=dot_data,feature_names=features,class_names=classes, filled=True, 
                     rounded=True, impurity=False)

graph = pydot.graph_from_dot_data(dot_data.getvalue())
graph[0].write_png('modelo.png')

ModuleNotFoundError: No module named 'pydot'

<img src="modelo.png">Modelo DecisionTree</img>